In [10]:
import pandas as pd
import pickle
from business_duration import businessDuration
import holidays
pd.set_option('display.max_columns', None)

In [11]:
import numpy as np
from scipy.stats import norm
def bs_form(S,r,vol,K,T,opt_type) :
    vol_sqrtT = vol*np.sqrt(T)
    d1 = ( np.log(S/K) + (r+0.5*vol**2)*T )/vol_sqrtT
    d2 = d1 - vol_sqrtT
    c = np.empty_like(S)
    cidx = (opt_type==1)
    pidx = ~cidx
    c[cidx] = S[cidx]*norm.cdf(d1[cidx]) - K[cidx]*np.exp(-r[cidx]*T[cidx])*norm.cdf(d2[cidx])
    c[pidx] = K[pidx]*np.exp(-r[pidx]*T[pidx])*norm.cdf(-d2[pidx]) - S[pidx]*norm.cdf(-d1[pidx])
    vega = S*norm.pdf(d1)*np.sqrt(T)
    return c,vega
def opt_bound(S,r,K,T,opt_type) :
    lower_bound = np.empty_like(S)
    upper_bound = np.empty_like(S)
    cidx = (opt_type==1)
    pidx = ~cidx
    lower_bound[cidx] = np.maximum(S[cidx]-K[cidx]*np.exp(-r[cidx]*T[cidx]),0.)
    lower_bound[pidx] = np.maximum(K[pidx]*np.exp(-r[pidx]*T[pidx])-S[pidx],0.)
    upper_bound[cidx] = S[cidx]
    upper_bound[pidx] = K[pidx]
    return lower_bound,upper_bound

In [12]:
with open("krx_data_3Y_new","rb") as fr:
    df_org = pickle.load(fr)
df = df_org.dropna(subset=['종가']).copy()
df = df.reset_index(drop=True)
df.head(2)

,today,종목코드,종목명,종가,대비,시가,고가,저가,내재변동성,익일정산가,거래량,거래대금,미결제약정,주식가격,분류,만기월,행사가,만기일,잔존만기,가격,rate,simple moneyness
0,2019-01-02,201P1220,코스피200 C 201901 220.0,37.85,-4.05,43.8,44.05,37.85,32.0,37.85,78,795,2010,258.23,1,201901,220.0,2019-01-10,0.027778,38.326301,1.964,1.173773
1,2019-01-02,201P1240,코스피200 C 201901 240.0,20.00,-2.00,21.5,21.50,20.00,3.0,20.00,2,10,106,258.23,1,201901,240.0,2019-01-10,0.027778,18.329979,1.964,1.075958


In [13]:
kvix=pd.read_excel('data_1752_20230313.xlsx')
kvix=kvix[['일자','종가']]
kvix['일자']=pd.to_datetime(kvix['일자'])
kvix=kvix.rename(columns={'종가':'kvix'})
kvix=kvix.set_index('일자',drop=True)
df=pd.merge(df,kvix,left_on='today',right_on='일자')
df.head(2)

C:\ProgramData\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,today,종목코드,종목명,종가,대비,시가,고가,저가,내재변동성,익일정산가,거래량,거래대금,미결제약정,주식가격,분류,만기월,행사가,만기일,잔존만기,가격,rate,simple moneyness,kvix
0,2019-01-02,201P1220,코스피200 C 201901 220.0,37.85,-4.05,43.8,44.05,37.85,32.0,37.85,78,795,2010,258.23,1,201901,220.0,2019-01-10,0.027778,38.326301,1.964,1.173773,19.74
1,2019-01-02,201P1240,코스피200 C 201901 240.0,20.00,-2.00,21.5,21.50,20.00,3.0,20.00,2,10,106,258.23,1,201901,240.0,2019-01-10,0.027778,18.329979,1.964,1.075958,19.74


In [14]:
c1 = df['today']
c2 = df['만기일']
holidays_kr = holidays.KR()
#T = businessDuration(startdate=c1[0],enddate=c2[0],holidaylist=holidays_kr,unit='day')/252.
n = len(df)
T = []
for i,(e1,e2) in enumerate(zip(c1,c2)) : 
    if i%(n//20)==0 : print('%3d%%'%round(100.*i/n),end=' ',flush=True)
    T.append( businessDuration(startdate=e1,enddate=e2,holidaylist=holidays_kr,unit='day')/252. )
print('')
T = np.array(T)
T

  0%   5%  10%  15%  20%  25%  30%  35%  40%  45%  50%  55%  60%  65%  70%  75%  80%  85%  90%  95% 100% 


array([0.02380952, 0.02380952, 0.02380952, ..., 2.90873016, 2.90873016,
       2.90873016])

In [15]:
df['잔존만기'] = T
df = df.drop(['simple moneyness','가격','종목코드','today','종목명','익일정산가','만기월','만기일'],axis=1)
df.head(2)

,종가,대비,시가,고가,저가,내재변동성,거래량,거래대금,미결제약정,주식가격,분류,행사가,잔존만기,rate,kvix
0,37.85,-4.05,43.8,44.05,37.85,32.0,78,795,2010,258.23,1,220.0,0.02381,1.964,19.74
1,20.00,-2.00,21.5,21.50,20.00,3.0,2,10,106,258.23,1,240.0,0.02381,1.964,19.74


In [16]:
S = df['주식가격']
r = 0.01*df['rate']
K = df['행사가']
T = df['잔존만기']
opt_type = df['분류']
lower_bound,upper_bound = opt_bound(S,r,K,T,opt_type)
df['이론하한'] = lower_bound
df.head(2)

,종가,대비,시가,고가,저가,내재변동성,거래량,거래대금,미결제약정,주식가격,분류,행사가,잔존만기,rate,kvix,이론하한
0,37.85,-4.05,43.8,44.05,37.85,32.0,78,795,2010,258.23,1,220.0,0.02381,1.964,19.74,38.332852
1,20.00,-2.00,21.5,21.50,20.00,3.0,2,10,106,258.23,1,240.0,0.02381,1.964,19.74,18.342202


In [17]:
check = np.sign(df['종가']-df['이론하한'])
idx = (check==1)

S = df['주식가격'][idx]
r = 0.01*df['rate'][idx]
K = df['행사가'][idx]
T = df['잔존만기'][idx]
opt_type = df['분류'][idx]
c = df['종가'][idx]

def imvol_f(vol) :
    c_bs,vega_bs = bs_form(S,r,vol,K,T,opt_type)
    return c_bs-c,vega_bs

a = np.full_like(S,1e-6)
b = np.full_like(S,5.)
for _ in range(20) :
    vol = (a+b)/2.
    delta_bs,vega_bs = imvol_f(vol)
    print( np.mean(delta_bs**2), end=' ')
    b[delta_bs>0]  = vol[delta_bs>0]
    a[delta_bs<=0] = vol[delta_bs<=0]
print('')

for _ in range(20) :
    delta_bs,vega_bs = imvol_f(vol)
    vol = vol-delta_bs/vega_bs
    print( np.mean(delta_bs**2), end=' ')  
print('')

df['이론변동성'] = None
df.loc[idx,'이론변동성'] = 100.*vol

14390.103313150274 3514.9764556596583 555.6887814348512 44.86530128491726 4.607102883185565 3.5477045516897703 0.7814625311487263 0.22417903269256304 0.058299438278006294 0.02009788972234801 0.010610133790540904 0.008216098257994901 0.007617516918108851 0.007469091172592197 0.007432041822266229 0.007422598415498835 0.007420250931458168 0.007419661353094917 0.007419514370646426 0.007419477932552085 
0.007419477932552085 0.1787314512032211 1.5422305449262033 0.09094467129049487 0.8601516284198031 4.4763223006284204e-10 1.3529808176345592e-14 1.2854711603072002e-23 3.3427404735334676e-28 3.3877037692467534e-28 3.325317938539984e-28 3.333471925181764e-28 3.3605281476790194e-28 3.327323054460236e-28 3.3312106776293944e-28 3.3561029900568664e-28 3.3335749529825742e-28 3.336404030843303e-28 3.3399243702638183e-28 3.34840783031742e-28 


In [18]:
df['이론변동성'] = df['이론변동성'].astype('float')
df['이상치수치'] = (df['이론변동성'] - df['내재변동성'])/df['내재변동성']*100.

In [19]:
df.sample(frac=1).query('(abs(이상치수치)>=20) or (이상치수치!=이상치수치)')

,종가,대비,시가,고가,저가,내재변동성,거래량,거래대금,미결제약정,주식가격,분류,행사가,잔존만기,rate,kvix,이론하한,이론변동성,이상치수치
70069,1.81,0.08,2.02,2.09,1.51,11.8,95500,43642,33158,295.31,1,295.0,0.047619,1.364,13.08,0.501547,6.019869,-48.984162
161806,103.00,-3.05,106.75,106.75,103.00,3.0,4,104,1456,416.88,1,315.0,0.047619,0.604,27.64,101.970587,74.777422,2392.580739
3436,65.20,3.85,65.10,65.20,65.10,16.0,4,65,6,273.88,1,210.0,2.845238,1.812,14.32,74.432346,NaN,NaN
5158,57.55,3.20,57.50,57.55,56.90,15.3,5,72,10,281.64,1,230.0,2.821429,1.855,14.94,63.368040,NaN,NaN
22109,16.40,3.05,13.75,17.50,13.75,8.0,38,158,124,283.70,0,300.0,0.035714,1.806,12.90,16.106562,19.177909,139.723859
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178396,11.10,-7.45,13.60,13.60,9.92,3.0,654,1822,1596,428.77,1,417.5,0.007937,0.450,20.00,11.284910,NaN,NaN
216119,29.95,-2.05,29.95,29.95,29.95,16.0,1,7,1757,395.40,1,365.0,0.043651,0.744,16.27,30.518519,NaN,NaN
87483,0.01,0.00,0.01,0.01,0.01,158.6,2,0,0,247.62,1,317.5,0.000000,1.305,43.07,0.000000,NaN,NaN
160031,11.40,-3.75,15.65,17.05,11.10,16.0,178,549,0,419.65,0,432.5,0.003968,0.532,26.43,12.840870,NaN,NaN


In [20]:
df.sample(frac=1).head(5)

,종가,대비,시가,고가,저가,내재변동성,거래량,거래대금,미결제약정,주식가격,분류,행사가,잔존만기,rate,kvix,이론하한,이론변동성,이상치수치
72625,0.67,-0.49,0.99,0.99,0.67,16.3,3849,716,4955,291.23,0,272.5,0.099206,1.407,15.08,0.000000,16.582286,1.731816
49202,0.01,0.00,0.01,0.02,0.01,14.6,415,1,2091,261.69,1,295.0,0.087302,1.529,15.97,0.000000,14.477482,-0.839162
171030,0.01,-0.02,0.03,0.03,0.01,26.0,10786,37,13071,424.85,1,457.5,0.011905,0.326,18.97,0.000000,24.507009,-5.742274
100458,37.60,-0.80,37.60,37.60,37.60,3.0,1,9,49,255.85,1,215.0,0.003968,0.759,25.75,40.856475,NaN,NaN
197547,0.27,0.04,0.28,0.28,0.27,14.0,202,10,9678,412.48,1,480.0,0.285714,0.558,15.62,0.000000,13.928884,-0.507975


In [22]:
df2 = df[['거래량','미결제약정','주식가격','잔존만기','rate','kvix','분류','이론하한','종가','이상치수치','행사가']].copy()
df2 = df2[df2['거래량']>=1]
#df2.dropna(how='any',inplace=True)
df2['주식가격'] /= df2['행사가']
df2['종가'] /= df2['행사가']

df2 = df2.rename(columns={'주식가격':'상대가격'})
df2 = df2.reset_index(drop=True)
df2.sample(frac=1)

,거래량,미결제약정,상대가격,잔존만기,rate,kvix,분류,이론하한,종가,이상치수치,행사가
94621,12,237,1.511813,0.424603,0.711,40.05,0,0.000000,0.009938,-2.928584,160.0
207933,116,794,1.526951,0.154762,0.671,17.03,0,0.000000,0.000272,-1.121007,257.5
176843,80996,14703,0.960823,0.027778,0.552,18.09,1,0.000000,0.000823,3.879883,437.5
103163,126,424,1.303951,0.126984,0.683,23.80,0,0.000000,0.001707,-1.490182,205.0
146542,13485,7043,1.002378,0.079365,0.535,21.27,1,1.037070,0.015892,-23.554954,370.0
...,...,...,...,...,...,...,...,...,...,...,...
116266,1,2,0.897832,0.063492,0.354,20.82,0,33.386399,0.099542,NaN,327.5
109890,8972,9071,1.375317,0.047619,0.435,31.73,0,0.000000,0.000439,-1.028576,205.0
28622,2,60,0.881333,0.785714,1.830,17.07,1,0.000000,0.007000,-12.290088,300.0
47655,627,4912,1.126667,0.111111,1.576,16.66,0,0.000000,0.001867,6.567396,225.0


In [23]:
with open("df.pkl","wb") as fr:
    pickle.dump(df2,fr)